In [5]:
import pandas as pd
import json
from sklearn.ensemble import RandomForestClassifier
import statsmodels.api as sm
import statsmodels.formula.api as smf 
from sklearn.model_selection import train_test_split
import numpy as np

In [6]:
data = pd.read_csv('../data/output.csv')
employees = pd.read_csv('../data/hrdata_OSF.csv')

employees_dict = {}
for emp in employees.to_dict('records'):
    name = emp['name']
    employees_dict[name] = {
        'acquired':emp['acquired'],
        'tenure':emp['tenure'],
        'title_status':emp['title_status'],
        'male':emp['male'],
        'rating':emp['rating'],
        'branch_id':emp['branch_id']
    }

retention = data.drop(columns=['Unnamed: 0','month','diff','n_emails','innov'])
reformation = data.drop(columns=['Unnamed: 0','month','diff','n_emails','ret'])

In [7]:
# retention = retention.to_dict('records')
retentions = {}
for r in retention.to_dict('records'):
    retentions[r['sender']] = r['ret']

In [8]:
reformations = {}
for r in reformation.to_dict('records'):
    reformations[r['sender']] = r['innov']

In [9]:
df = []
for emp in employees_dict:
    if emp in retentions and emp in reformations:

        temp = employees_dict[emp]
        temp['ret'] = retentions[emp]
        temp['innov'] = reformations[emp]
        temp['name'] = emp

        df.append(employees_dict[emp])

In [10]:
df = pd.DataFrame(df)
df.fillna(0, inplace=True)
X_train, X_test= train_test_split(df, test_size=0.33, random_state=42)

In [11]:
retentions_X_train = X_train.drop(columns=['innov','name'], axis=1)
reformations_X_train = X_train.drop(columns=['ret','name'], axis=1)

retentions_X_train = sm.add_constant(retentions_X_train) 

retentions_y_train = X_train['ret']
reformations_y_train = X_train['innov']

In [12]:
retentions_X_train

,const,acquired,tenure,title_status,male,rating,branch_id,ret
69,1.0,0,17.645448,2,1,0.000,3,0.91
127,1.0,0,4.098563,0,0,0.000,10,0.83
27,1.0,0,4.599589,0,0,3.460,3,0.91
150,1.0,0,1.752225,0,1,3.964,47,0.75
124,1.0,0,4.599589,1,0,0.000,10,0.83
...,...,...,...,...,...,...,...,...
71,1.0,0,5.796030,0,0,3.292,47,0.83
106,1.0,0,12.199863,2,1,4.040,47,0.90
14,1.0,0,3.457906,0,1,0.000,5,0.92
92,1.0,0,7.460643,1,1,2.720,18,0.95


In [13]:
retentions_X_test = X_test.drop(columns=['innov','name','ret'], axis=1)
reformations_X_test = X_test.drop(columns=['ret','name','innov'], axis=1)

# retentions_X_test = sm.add_constant(retentions_X_test) 

retentions_y_test = X_test['ret']
reformations_y_test = X_test['innov']

In [14]:
retentions_y_test.to_list()

[0.91,
 0.64,
 0.9,
 0.88,
 0.87,
 0.91,
 0.94,
 0.94,
 0.93,
 0.86,
 0.91,
 0.85,
 0.9,
 0.85,
 0.87,
 0.91,
 0.9,
 0.91,
 0.88,
 0.86,
 0.57,
 0.85,
 0.87,
 0.94,
 0.89,
 0.86,
 0.89,
 0.89,
 0.93,
 0.71,
 0.93,
 0.88,
 0.93,
 0.94,
 0.94,
 0.91,
 0.89,
 0.94,
 0.91,
 0.89,
 0.9,
 0.87,
 0.96,
 0.93,
 0.93,
 0.99,
 0.9,
 0.8,
 0.95,
 0.9,
 0.88,
 0.91,
 0.91]

In [15]:
# olsmod = sm.OLS(np.asarraya(retentions_X_train),retentions_y_train)
# olsres = olsmod.fit()
olsmod = smf.ols(formula='ret ~ acquired + tenure + title_status + male + rating + branch_id', data=retentions_X_train)
olsres = olsmod.fit()

In [16]:
olsmod_innov = smf.ols(formula='innov ~ acquired + tenure + title_status + male + rating + branch_id', data=reformations_X_train)
olsres_innov = olsmod_innov.fit()

In [17]:
olsres.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ret   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.037
Method:                 Least Squares   F-statistic:                     1.800
Date:                Tue, 09 Aug 2022   Prob (F-statistic):              0.120
Time:                        11:55:18   Log-Likelihood:                 143.31
No. Observations:                 106   AIC:                            -274.6
Df Residuals:                     100   BIC:                            -258.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.8659      0.014     62.667      0.000       0.839       0.893
acquired     -6.382e-17    6.9e-17     -0.925      0.357   -2.01e-16    7.31e-17
tenure           0.0032      0.002      2.124      0.036       0.000       0.006
title_status     0.0069      0.010      0.683      0.496      -0.013       0.027
male             0.0092      0.014      0.647      0.519      -0.019       0.038
rating          -0.0008      0.004     -0.211      0.834      -0.009       0.007
branch_id        0.0002      0.000      0.472      0.638      -0.001       0.001
==============================================================================
Omnibus:                       92.526   Durbin-Watson:                   1.938
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              981.075
Skew:                          -2.806   Prob(JB):                    9.17e-214
Kurtosis:                      16.807   Cond. No.                     8.75e+17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 7.06e-32. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [18]:
ypred = olsres.predict(retentions_X_test)
ypred

78     0.896198
155    0.879077
128    0.885943
55     0.879431
94     0.945199
29     0.872755
147    0.872397
51     0.884128
98     0.896122
141    0.889138
19     0.878437
60     0.921012
15     0.881170
65     0.918064
24     0.890907
30     0.888784
126    0.879759
101    0.915175
96     0.897889
16     0.877261
151    0.885900
18     0.887871
12     0.896598
9      0.896975
31     0.887834
125    0.889317
95     0.885469
56     0.886583
145    0.895466
152    0.880153
135    0.887752
76     0.884190
75     0.904599
138    0.883252
2      0.881504
86     0.877134
45     0.896771
42     0.887564
68     0.877899
118    0.873087
26     0.909201
137    0.939308
146    0.879329
90     0.903859
66     0.909875
36     0.889378
82     0.897069
22     0.882257
85     0.888540
81     0.882154
112    0.926373
11     0.884080
109    0.881504
dtype: float64

In [19]:
ypred_innov = olsres_innov.predict(reformations_X_test)
ypred_innov

78     0.004251
155    0.003914
128    0.007771
55     0.007518
94    -0.003771
29     0.008338
147    0.010028
51     0.009755
98     0.002022
141    0.004171
19     0.004333
60    -0.001832
15     0.006171
65     0.000119
24     0.000418
30     0.007158
126    0.003298
101   -0.002585
96     0.002117
16     0.008076
151    0.001608
18     0.008991
12     0.000148
9      0.006048
31     0.007603
125    0.006921
95     0.007113
56     0.002887
145    0.009034
152    0.003637
135   -0.000036
76     0.009739
75     0.000133
138    0.004502
2      0.007583
86     0.008509
45     0.000257
42     0.003809
68     0.008112
118    0.008652
26    -0.003315
137   -0.012044
146    0.010991
90     0.000890
66    -0.004292
36     0.005656
82     0.000634
22     0.006990
85     0.000435
81     0.008466
112   -0.007537
11     0.002828
109    0.007583
dtype: float64

In [20]:
reformations_X_test

,acquired,tenure,title_status,male,rating,branch_id
78,0,5.664613,1,0,0.000,27
155,0,1.333333,0,0,0.000,47
128,0,2.652977,2,0,3.560,3
55,0,3.605749,0,0,0.000,10
94,0,17.259411,2,1,0.000,3
29,0,1.007529,0,0,0.000,19
147,0,1.831622,0,0,0.000,3
51,0,3.321013,1,0,0.000,3
98,0,8.084873,0,0,0.000,22
141,0,2.362765,1,0,0.000,46
